<a href="https://colab.research.google.com/github/coll-j/B21-CAP0001/blob/ml/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [78]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re
import nltk

from tensorflow.keras.layers import Dense, RNN, LSTM, Embedding

from gensim.models.wrappers import FastText
from google.colab import drive

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
drive.mount('/content/drive')

Mounted at /content/drive


# Downloads

### Dataset

In [2]:
!wget https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/re_dataset.csv

--2021-05-27 06:25:57--  https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/re_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1858473 (1.8M) [text/plain]
Saving to: ‘re_dataset.csv’

re_dataset.csv      100%[===================>]   1.77M  --.-KB/s    in 0.08s   

2021-05-27 06:25:58 (23.1 MB/s) - ‘re_dataset.csv’ saved [1858473/1858473]



### Kamus Alay

In [3]:
!wget https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv

--2021-05-27 06:27:48--  https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285941 (279K) [text/plain]
Saving to: ‘new_kamusalay.csv’

new_kamusalay.csv   100%[===================>] 279.24K  --.-KB/s    in 0.01s   

2021-05-27 06:27:48 (21.2 MB/s) - ‘new_kamusalay.csv’ saved [285941/285941]



# Read Data

In [46]:
df = pd.read_csv('re_dataset.csv', encoding="ISO-8859-1")
df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [72]:
num_class = len(df.columns) - 1

In [ ]:
df['Tweet'][5]

"USER Ya bani taplak dkk \\xf0\\x9f\\x98\\x84\\xf0\\x9f\\x98\\x84\\xf0\\x9f\\x98\\x84'"

In [ ]:
df['Tweet'][8]

'Nah admin belanja satu lagi port terbaik nak makan Ais Kepal Milo, Ais Kepal Horlicks atau Cendol Toping kaw kaw. ð??\xad; ; Doket mano tu ? Gerai Rojak Mertuaku - Taipan 2 (depan TWINS BABY &amp; ROMANTIKA / Bank Islam Senawang) ð???'

In [ ]:
len(df)

13169

# Pre-Processing

In [47]:
kamus_alay = pd.read_csv('new_kamusalay.csv', encoding="ISO-8859-1", header=None)
kamus_alay.head()

,0,1
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [48]:
kamus_alay_dict = {}
for i, row in kamus_alay.iterrows():
  kamus_alay_dict[row[0]] = row[1]

kamus_alay_dict

{'anakjakartaasikasik': 'anak jakarta asyik asyik',
 'pakcikdahtua': 'pak cik sudah tua',
 'pakcikmudalagi': 'pak cik muda lagi',
 't3tapjokowi': 'tetap jokowi',
 '3x': 'tiga kali',
 'aamiin': 'amin',
 'aamiinn': 'amin',
 'aamin': 'amin',
 'aammiin': 'amin',
 'abis': 'habis',
 'abisin': 'habiskan',
 'acau': 'kacau',
 'achok': 'ahok',
 'ad': 'ada',
 'adek': 'adik',
 'adl': 'adalah',
 'adlah': 'adalah',
 'adlh': 'adalah',
 'ado': 'ada',
 'aduhh': 'aduh',
 'aer': 'air',
 'afdol': 'afdal',
 'agamaataualqur': 'agama alquran',
 'agm': 'agama',
 'agma': 'agama',
 'ahaha': 'haha',
 'ahahaha': 'haha',
 'ahehehehe': 'hehe',
 'ahir': 'akhir',
 'ahirnya': 'akhirnya',
 'ahk': 'ahok',
 'ahlamdulillah': 'alhamdulillah',
 'ahli2': 'para ahli',
 'ahlusunnah': 'ahlus sunah',
 'ahmaddani': 'ahmad dhani',
 'aho': 'ahok',
 'ahoax': 'ahok',
 'ahoaxx': 'ahok',
 'ahog': 'ahok',
 'ahokataudjarot': 'ahok djarot',
 'ahokbebanijokowi': 'ahok beban jokowi',
 'ahokbtp': 'ahok basuki tjahaja purnama',
 'ahokditolakw

## Cleaning text

In [49]:
tweets = df['Tweet'].str.lower()
tweets

0        - disaat semua cowok berusaha melacak perhatia...
1        rt user: user siapa yang telat ngasih tau elu?...
2        41. kadang aku berfikir, kenapa aku tetap perc...
3        user user aku itu aku\n\nku tau matamu sipit t...
4        user user kaum cebong kapir udah keliatan dong...
                               ...                        
13164    user jangan asal ngomong ndasmu. congor lu yg ...
13165                         user kasur mana enak kunyuk'
13166    user hati hati bisu :( .g\n\nlagi bosan huft \...
13167    user user user user bom yang real mudah terdet...
13168    user mana situ ngasih(": itu cuma foto ya kuti...
Name: Tweet, Length: 13169, dtype: object

In [28]:
tweets[0]

"- disaat semua cowok berusaha melacak perhatian gue. loe lantas remehkan perhatian yg gue kasih khusus ke elo. basic elo cowok bego ! ! !'"

In [50]:
clean_tweets = tweets
query = r"http\S+| url" # hilangkan link
clean_tweets = list(map(lambda s: re.sub(query, '', s), clean_tweets))
query = r'user[:]? |\\[\\A-Za-z0-9]*|rt ' # Hilangkan 'USER', RT, dan encoding emoji ex: '\\xf0\\xxx\\xxx...'
clean_tweets = list(map(lambda s: re.sub(query, '', s), clean_tweets))
query = r'[^a-zA-Z\d\s]' # ambil alphanumeric saja
clean_tweets = list(map(lambda s: re.sub(query, ' ', s), clean_tweets))
clean_tweets = list(map(lambda s: ' '.join([kamus_alay_dict.get(w, w) for w in s.split()]), clean_tweets))
query = r'(?:^| )\w(?:$| )'
clean_tweets = list(map(lambda s: re.sub(query, ' ', s), clean_tweets)) # hilangkan single letter
query = r" +"
clean_tweets = list(map(lambda s: re.sub(query, ' ', s.strip()), clean_tweets)) # hilangkan trailing spaces
clean_tweets

['di saat semua cowok berusaha melacak perhatian gue kamu lantas remehkan perhatian yang gue kasih khusus ke kamu basic kamu cowok bego',
 'siapa yang telat memberi tau kamu edan sarap gue bergaul dengan cigax jifla calis sama siapa itu licew juga',
 '41 kadang aku berpikir kenapa aku tetap percaya pada tuhan padahal aku selalu jatuh berkali kali kadang aku merasa tuhan itu meninggalkan aku sendirian ketika orang tuaku berencana berpisah ketika kakakku lebih memilih jadi kristen ketika aku anak ter',
 'aku itu aku tau matamu sipit tapi dilihat dari mana itu aku',
 'kaum cebong kafir sudah kelihatan dongoknya dari awal tambah dungu lagi haha',
 'ya bani taplak dan kawan kawan',
 'deklarasi pilihan kepala daerah 2018 aman dan anti hoaks warga dukuh sari jabon',
 'gue baru saja selesai re watch aldnoah zero paling kampret memang akhirnya karakter utama cowoknya kena friendzone bro xd',
 'nah admin belanja satu lagi poterbaik nak makan ais kepal milo ais kepal horlicks atau cendol toping k

## Tokenize
+ To sequence and padding

In [51]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True)
tokenizer.fit_on_texts(clean_tweets)
padded_sequence = tokenizer.texts_to_sequences(clean_tweets)
padded_sequence

[[2,
  157,
  67,
  436,
  1007,
  7147,
  1365,
  17,
  6,
  2453,
  7148,
  1365,
  1,
  17,
  160,
  205,
  30,
  6,
  7149,
  6,
  436,
  273],
 [134,
  1,
  2263,
  592,
  89,
  6,
  578,
  519,
  17,
  4045,
  15,
  10229,
  10230,
  10231,
  19,
  134,
  5,
  10232,
  23],
 [4705,
  690,
  27,
  1183,
  115,
  27,
  145,
  294,
  57,
  261,
  247,
  27,
  176,
  1008,
  3200,
  174,
  690,
  27,
  332,
  261,
  5,
  1222,
  27,
  3201,
  377,
  11,
  10233,
  5631,
  10234,
  377,
  10235,
  60,
  337,
  20,
  78,
  377,
  27,
  90,
  2264],
 [27, 5, 27, 89, 1223, 179, 41, 1796, 21, 105, 5, 27],
 [260, 55, 82, 13, 487, 10236, 21, 603, 933, 181, 36, 109],
 [16, 202, 673, 3, 818, 818],
 [734, 88, 70, 52, 114, 408, 3, 126, 210, 194, 5632, 2133, 1366],
 [17,
  71,
  14,
  501,
  4046,
  10237,
  10238,
  7150,
  165,
  156,
  53,
  480,
  1890,
  934,
  3202,
  379,
  10239,
  401,
  4706],
 [579,
  849,
  1009,
  111,
  36,
  10240,
  593,
  199,
  7151,
  7152,
  10241,
  7151,
  

In [53]:
max_len = max([len(seq) for seq in padded_sequence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(padded_sequence, maxlen=max_len)
padded_sequence # This will be the input

array([[    0,     0,     0, ...,     6,   436,   273],
       [    0,     0,     0, ...,     5, 10232,    23],
       [    0,     0,     0, ...,    27,    90,  2264],
       ...,
       [    0,     0,     0, ...,     4,  1725,  3309],
       [    0,     0,     0, ...,   630,  1835,  1018],
       [    0,     0,     0, ...,    16,  2057,   146]], dtype=int32)

In [75]:
padded_sequence.shape

(13169, 58)

## Output

In [84]:
y = df[df.columns[1:]].to_numpy()

# Load FastText Word Embedding

In [54]:
!tar -xvzf "drive/My Drive/Colab Notebooks/Capstone/id.tar.gz"

id.bin
id.vec


In [55]:
embedding_path = 'id.bin'
embedding_model = FastText.load_fasttext_format(embedding_path)

In [62]:
embedding_model.most_similar('guru')

[('pengajar', 0.6640160083770752),
 ('dosen', 0.6254853010177612),
 ('murid', 0.6231797337532043),
 ('guru-guru', 0.6127239465713501),
 ('mengajar', 0.5958520770072937),
 ('pendidik', 0.565003514289856),
 ('gurunya', 0.538303792476654),
 ('belajar', 0.5202033519744873),
 ('honorer', 0.5138474106788635),
 ('diajar', 0.5089484453201294)]

In [ ]:
embedding_model.most_similar('ahok')

[('tjahaja', 0.6268017292022705),
 ('jokowi', 0.6124308705329895),
 ('prabowo', 0.5613927245140076),
 ('bowo', 0.5557748675346375),
 ('basuki', 0.5413249731063843),
 ('subianto', 0.5392811298370361),
 ('dki', 0.5362879037857056),
 ('widodo', 0.5347503423690796),
 ('sby', 0.5129902362823486),
 ('joko', 0.5028191804885864)]

In [63]:
embedding_model.most_similar('jokowi')

[('joko', 0.6402931809425354),
 ('sby', 0.6383484601974487),
 ('widodo', 0.6331063508987427),
 ('ahok', 0.6124308705329895),
 ('bowo', 0.6094229221343994),
 ('kalla', 0.5852698087692261),
 ('megawati', 0.5805439352989197),
 ('capres', 0.5768967270851135),
 ('prabowo', 0.5698248147964478),
 ('yudhoyono', 0.5645459890365601)]

In [67]:
embedding_matrix_ft = np.random.random((len(tokenizer.word_index) + 1, embedding_model.vector_size))

oov = 0
for word, i in tokenizer.word_index.items():
  try:
    embedding_matrix_ft[i] = ft.wv[word]
  except:
    oov +=1

oov, embedding_matrix_ft

(19540,
 array([[0.55305256, 0.46756272, 0.97691393, ..., 0.06780521, 0.88853288,
         0.44522017],
        [0.27331015, 0.57209803, 0.04780432, ..., 0.14916147, 0.30113583,
         0.96325851],
        [0.6836368 , 0.43613605, 0.02147147, ..., 0.15824695, 0.9701627 ,
         0.754029  ],
        ...,
        [0.12695929, 0.27936995, 0.85331921, ..., 0.71773257, 0.21747471,
         0.64962179],
        [0.02551859, 0.03188309, 0.87577282, ..., 0.02260644, 0.99094453,
         0.57866469],
        [0.82173385, 0.3929426 , 0.26055717, ..., 0.12191846, 0.92731099,
         0.337065  ]]))

# Build Model

In [80]:
model = tf.keras.Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 
                    embedding_model.vector_size,
                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix_ft),
                    trainable=False
                    ))
model.add(LSTM(32))
model.add(Dense(num_class, activation='sigmoid'))

In [81]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build((None, 58))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         5862300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dense_1 (Dense)              (None, 12)                396       
Total params: 5,905,320
Trainable params: 43,020
Non-trainable params: 5,862,300
_________________________________________________________________


In [86]:
model.fit(padded_sequence[:5, :], y[:5, :], epochs=3) # trial

Epoch 1/3
1/1 [==============================] - 0s 20ms/step - loss: 0.5693 - accuracy: 0.2000
Epoch 2/3
1/1 [==============================] - 0s 19ms/step - loss: 0.5427 - accuracy: 0.2000
Epoch 3/3
1/1 [==============================] - 0s 17ms/step - loss: 0.5227 - accuracy: 0.2000
